In [2]:
# ! pip install sentence-transformers
# ! pip install git+https://github.com/huggingface/transformers     
# ! pip install tf-models-official    
# ! pip install pandas-profiling==2.7.1 
# ! pip install surprise

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ast
import operator
import sentence_transformers
from sentence_transformers import SentenceTransformer
# from scipy import sparse
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.model_selection import GridSearchCV
# import tensorflow_hub as hub
# import tensorflow_datasets as tfds
import networkx as nx
import pandas_profiling
from pandas_profiling import ProfileReport
from collections import defaultdict
# from official.modeling import tf_utils
# from official import nlp
# from official.nlp import bert
# model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')
# from scipy.sparse import csr_matrix
# from sklearn.metrics.pairwise import pairwise_distances 
# from sklearn.metrics import make_scorer
# from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import accuracy
from surprise import NormalPredictor
from surprise.prediction_algorithms import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import train_test_split
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
# Load additional submodules
# import official.nlp.optimization
# import official.nlp.bert.bert_models
# import official.nlp.bert.configs
# import official.nlp.bert.run_classifier
# import official.nlp.bert.tokenization
# import official.nlp.data.classifier_data_lib
# import official.nlp.modeling.losses
# import official.nlp.modeling.models
# import official.nlp.modeling.networks


In [5]:
# Import in Colabs
# interactions = pd.read_csv('/content/drive/MyDrive/Recipe_Data/RAW_interactions.csv', engine='python')
# recipes = pd.read_csv('/content/drive/MyDrive/Recipe_Data/RAW_recipes.csv', engine='python')
# full = pd.merge(recipes, interactions, right_on=['recipe_id'], left_on=['id']) 

# Import in VS code
interactions = pd.read_csv('./Data (remove before  commit)/RAW_interactions.csv')
recipes = pd.read_csv('./Data (remove before  commit)/RAW_recipes.csv')
# # movies.set_index('movie_id',inplace=True)

In [28]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226657 entries, 0 to 231636
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            226657 non-null  object
 1   id              226657 non-null  int64 
 2   minutes         226657 non-null  int64 
 3   contributor_id  226657 non-null  int64 
 4   submitted       226657 non-null  object
 5   tags            226657 non-null  object
 6   nutrition       226657 non-null  object
 7   n_steps         226657 non-null  int64 
 8   steps           226657 non-null  object
 9   description     226657 non-null  object
 10  ingredients     226657 non-null  object
 11  n_ingredients   226657 non-null  int64 
 12  instructions    226657 non-null  object
dtypes: int64(5), object(8)
memory usage: 24.2+ MB


## Interactions

In [7]:
#inspecting data

# interactions.describe

interactions.isnull().sum() 


#examine ratings
interactions['rating'].value_counts()
rating_look = interactions.groupby('recipe_id')['rating'].mean().sort_values(ascending=False)
rating_sum = interactions.groupby('recipe_id')['rating'].sum().sort_values(ascending=False)
rating_look.value_counts().head(20).sort_values(ascending=False)

unique = interactions['recipe_id'].unique()

In [8]:


# data = interactions['rating'].value_counts().sort_index(ascending=False)
# trace = go.Bar(x = data.index,
#                text = ['{:.1f} %'.format(val) for val in (data.values / interactions.shape[0] * 100)],
#                textposition = 'auto',
#                textfont = dict(color = '#000000'),
#                y = data.values,
#                )
# # Create layout
# layout = dict(title = 'Distribution Of {} Recipes'.format(interactions.shape[0]),
#               xaxis = dict(title = 'Rating'),
#               yaxis = dict(title = 'Count'))
# # Create plot
# fig = go.Figure(data=[trace], layout=layout)
# iplot(fig)

In [9]:
# # Number of ratings per book
# data = interactions.groupby(str('recipe_id'))['rating'].count()

# # Create trace
# trace = go.Histogram(x = data.values,
#                      name = 'Ratings',
#                      xbins = dict(start = 0,
#                                   end = 50,
#                                   size = 2))
# # Create layout
# layout = go.Layout(title = 'Distribution Of Number of Ratings Per Recipe,
#                    xaxis = dict(title = 'Number of Ratings Per Recipe'),
#                    yaxis = dict(title = 'Count'),
#                    bargap = 0.2)

# # Create plot
# fig = go.Figure(data=[trace], layout=layout)
# iplot(fig)

In [10]:
#quick fancy EDA from Pandas

# pandas_profiling.ProfileReport(interactions)


In [11]:
#cleaning data

interactions.dropna(inplace=True)
interactions=interactions.drop(columns = ['date', 'review'])

In [12]:



#trim users with 20 or less reviews
unum_reviews = interactions['rating'].groupby([interactions.user_id]).agg('count').sort_values(ascending = True)
unum_reviews2 = unum_reviews[unum_reviews >20]
interactions = interactions[interactions['user_id'].isin(unum_reviews2.index)]

# trim recipes with 50 or less reviews
rnum_reviews = interactions['rating'].groupby([interactions.recipe_id]).agg('count').sort_values(ascending = True)
rnum_reviews2 = rnum_reviews[rnum_reviews > 50]
interactions = interactions[interactions['recipe_id'].isin(rnum_reviews2.index)]
# remove zeros for NMF
MNF_data=interactions[interactions.rating > 0]


## Recipes

In [13]:
#inspecting data

# recipes.info()

# recipes= recipes.fillna(value=np.nan)
# # recipes.isnull().sum() 
# recipes['id'].value_counts()

# unique = recipes['recipe_id'].unique()




In [14]:
# cleaning data
recipes= recipes.fillna(value=np.nan)
recipes.dropna(inplace=True)

# Convert steps for making recipe from a list to a string

recipes['instructions'] = recipes['steps'].apply(lambda x : " ".join(ast.literal_eval(x)))
recipes['ingredients']=recipes['ingredients'].apply(lambda x : " ".join(ast.literal_eval(x)))


In [15]:
# recipes

In [16]:
# pandas_profiling.ProfileReport(recipes)

In [17]:
def rmse_vs_factors(algorithm, data):
  """Returns: rmse_algorithm i.e. a list of mean RMSE of CV = 5 in cross_validate() for each  factor k in range(1, 101, 1)
  100 values 
  Arg:  i.) algorithm = Matrix factoization algorithm, e.g SVD/NMF/PMF, ii.)  data = surprise.dataset.DatasetAutoFolds
  """
  
  rmse_algorithm = []
  
  for k in range(1, len(data), 1):
    algo = algorithm(n_factors = k)
    
    #["test_rmse"] is a numpy array with min accuracy value for each testset
    loss_fce = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)["test_rmse"].mean() 
    rmse_algorithm.append(loss_fce)
  
  return rmse_algorithm

In [18]:
def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


### Data All Together After Cleaning For Possible Use Later

In [19]:

#interactions with recipes for final reccomendations
full_data = pd.merge(recipes, interactions, right_on=['recipe_id'], left_on=['id']) 


## Similarity Recommender Model Based on Ratings

In [20]:

reader = Reader(rating_scale= (1,5) )
df = pd.DataFrame(full_data, columns = ['user_id', 'name', 'rating'])
data = Dataset.load_from_df(df, reader)


###  SVD - Singular Value Decomposition

In [21]:
algo = SVD()
SVD_model = cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8837  0.8940  0.8874  0.8904  0.9004  0.8912  0.0057  
MAE (testset)     0.5129  0.5156  0.5119  0.5131  0.5193  0.5146  0.0027  
Fit time          3.79    3.87    3.93    3.99    3.80    3.88    0.08    
Test time         0.10    0.27    0.10    0.10    0.11    0.13    0.07    


In [22]:
# First train an SVD algorithm on dataset.

trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# prediction on all pairs
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    recos = {}
    print(uid, [iid for (iid, _) in user_ratings])

112080 ['2  handed kitchen sink tomato sandwich', 'almost apple pie', 'alton brown s baked brown rice', 'best baked potato', 'best banana bread']
29956 ['never weep  whipped cream', '2  handed kitchen sink tomato sandwich', '4 minute spicy garlic shrimp', 'amazing chicken marinade', 'balsamic tomato couscous']
57828 ['whatever floats your boat  brownies', '2  handed kitchen sink tomato sandwich', 'apple bites', 'beth s melt in your mouth barbecue ribs  oven', 'big grandma s best peanut butter cookies']
87024 ['shrimp  cocktail sauce', '2  handed kitchen sink tomato sandwich', '4 minute spicy garlic shrimp', 'alton brown s baked brown rice', 'angel food cake   homemade']
38730 ['never weep  whipped cream', 'ant  kelly s london broil marinade', 'whatever floats your boat  brownies', 'shrimp  cocktail sauce', '2  handed kitchen sink tomato sandwich']
153750 ['ant  kelly s london broil marinade', '2  handed kitchen sink tomato sandwich', 'allrighty then scrambled eggs   paula deen', 'alton

algo = SVD()
SVD_model = cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True) 

In [23]:
# algo_SVD = SVD(n_factors = 11)
# algo_SVD.fit(trainset)


# # Predict ratings for all pairs (i,j) that are NOT in the training set.
# testset = trainset.build_anti_testset()

# predictions = algo_SVD.test(testset)

###  SVD - Singular Value Decomposition ++

In [24]:
# algo2 = SVDpp()
# SVDpp_model = cross_validate(algo2, data, measures=['RMSE','MAE'], cv=5, verbose=True)



Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1 |Fold 2 |Fold 3  |Fold 4  |Fold 5  |Mean    |   Std     
RMSE (testset)    1.2266 |1.2273 |  1.2190|  1.2255|  1.2271|  1.2251|  0.0031  
MAE (testset)     0.7356 |0.7368 |  0.7325|  0.7362|  0.7364|  0.7355|  0.0016  
Fit time          5481.10|7345.52| 5179.65|32823.22| 7417.45|11649.39|10627.03
Test time|         93.09 | 74.48 |   91.17| 84.21  |  110.41|   90.67|   11.83   

overall time was 58723 seconds

This took way too long with no sizeable payoff

Non-Negative Matrix Factorization (NMF)

In [25]:
reader = Reader(rating_scale= (1,5) )
df2 = pd.DataFrame(MNF_data, columns = ['user_id', 'name', 'rating'])
NMFdata = Dataset.load_from_df(df2, reader)

Ntrainset = NMFdata.build_full_trainset()

Ntestset = trainset.build_anti_testset()

algo3 = NMF(biased=True)
NMF_model = cross_validate(algo3, NMFdata, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8052  0.8065  0.8037  0.8015  0.8112  0.8056  0.0032  
MAE (testset)     0.6954  0.6950  0.6928  0.6917  0.7005  0.6951  0.0030  
Fit time          8.86    9.28    9.40    9.01    9.12    9.13    0.19    
Test time         39.93   77.45   45.95   33.76   39.10   47.24   15.59   


In [26]:
# algo3 = NMF(biased=True)
# NMF_model = cross_validate(algo3, NMFdata, measures=['RMSE','MAE'], cv=5, verbose=True)

In [27]:
# rmse_nmf = rmse_vs_factors(NMF, NMFdata)
# trainset = NMFdata.build_full_trainset()
# param_grid = {'n_factors': [11,14,15,16,17,18,20]}
# gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=5)
# gs.fit(NMFdata)



# # best RMSE score
# print(gs.best_score['rmse'])

# # combination of parameters that gave the best RMSE score
# print(gs.best_params['rmse'])

# # 0.8861525979842921
# # {'n_factors': 17}

# algo_NMF = NMF(n_factors = 16,biased=True )
# algo_NMF.fit(trainset)


# # Predict ratings for all pairs (u, i) that are NOT in the training set.
# testset = trainset.build_anti_testset()
# predictions = algo_NMF.test(testset)

# # ist_NMF_124, pred_NMF_124 = get_top_n(predictions, recipe_id = NMFdata.recipe_id, userId = 124, original_ratings = NMFdata.rating)

# # takes long time 4348

TypeError: object of type 'DatasetAutoFolds' has no len()

In [ ]:
# hlist_NMF_124, pred_NMF_124 = get_top_n(predictions, recipe_id = NMFdata.recipe_id, userId = 124, original_ratings = NMFdata.rating)

AttributeError: 'DatasetAutoFolds' object has no attribute 'recipe_id'

In [29]:
# First train an SVD algorithm on the movielens dataset.

trainset = NMFdata.build_full_trainset()
algo3 = NMF()
# algo3.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
# testset = trainset.build_anti_testset()
predictions = algo3.test(testset)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
# algo.predict(2,1214,1) #SVD prediciton
# algo3.predict(2,1214,4) #NMF prediction
# # algo2.predict(2,1214,4) #SVD++ prediction

In [ ]:

def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n




In [ ]:
def get_top_reco(predictions, userId, name, rating, n=5):
    '''Return the top N (default) movieId for a user,.i.e. userID and history for comparisom
    Args:
    Returns: 
  
    '''
    #Peart I.: Surprise docomuntation
    
    #1. First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    #2. Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[: n ]
    
    #Part II.: inspired by: https://beckernick.github.io/matrix-factorization-recommender/
    
    #3. Tells how many movies the user has already rated
    user_data = df.rating[df.user_id == ('user_id')]
    print('User {0} has already rated {1} recipes.'.format(userId, user_data.shape[0]))

    
    #4. Data Frame with predictions. 
    preds_df = pd.DataFrame([(id, pair[0],pair[1]) for id, row in top_n.items() for pair in row],
                        columns=["userId" ,"name","pred"])
    
    
    #5. Return pred_usr, i.e. top N recommended movies with (merged) titles and genres. 
    # pred_usr = preds_df[preds_df["userId"] == (userId)].merge(df, how = 'left', left_on = 'name', right_on = 'name')
            
    #6. Return hist_usr, i.e. top N historically rated movies with (merged) titles and genres for holistic evaluation
    hist_usr = ratings_df[ratings_df.userId == (userId) ].sort_values("rating", ascending = False).merge\
    (df, how = 'left', left_on = 'name', right_on = 'name')
    
    
    return hist_usr, pred_usr

In [ ]:
# trainset = data.build_full_trainset()
# algo = SVD()
# algo.fit(trainset)

# # prediction on all pairs
# testset = trainset.build_anti_testset()
# predictions = algo.test(testset)

# top_reco = get_top_reco(predictions, df.name, df.rating, df.user_id, n=5)

# # Print the recommended items for each user
# for uid, user_ratings in top_reco.items():
#     recos = {}
#     print(uid, [iid for (iid, _) in user_ratings])

User 0        how i got my family to eat spinach  spinach ca...
1        how i got my family to eat spinach  spinach ca...
2        how i got my family to eat spinach  spinach ca...
3        how i got my family to eat spinach  spinach ca...
4        how i got my family to eat spinach  spinach ca...
                               ...                        
87620                  zucchini or yellow squash casserole
87621                  zucchini or yellow squash casserole
87622                  zucchini or yellow squash casserole
87623                  zucchini or yellow squash casserole
87624                  zucchini or yellow squash casserole
Name: name, Length: 87625, dtype: object has already rated 0 recipes.


NameError: name 'ratings_df' is not defined

In [ ]:
def rmse_vs_factors(algorithm, data):
  """Returns: rmse_algorithm i.e. a list of mean RMSE of CV = 5 in cross_validate() for each  factor k in range(1, 101, 1)
  100 values 
  Arg:  i.) algorithm = Matrix factoization algorithm, e.g SVD/NMF/PMF, ii.)  data = surprise.dataset.DatasetAutoFolds
  """
  
  rmse_algorithm = []
  
  for k in range(1, 101, 1):
    algo = algorithm(n_factors = k)
    
    #["test_rmse"] is a numpy array with min accuracy value for each testset
    loss_fce = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)["test_rmse"].mean() 
    rmse_algorithm.append(loss_fce)
  
  return rmse_algorithm

In [ ]:
Ntrainset = NMFdata.build_full_trainset()

Ntestset = trainset.build_anti_testset()

In [ ]:
param_grid = {'n_factors': [11,14,15,16,17,18,20]}
gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=5)
gs.fit(NMFdata)



# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.625879072460324
{'n_factors': 20}


In [ ]:
algo_NMF = NMF(n_factors = 16)
algo_NMF.fit(Ntrainset)


# Predict ratings for all pairs (u, i) that are NOT in the training set.
testset = Ntrainset.build_anti_testset()
predictions = algo_NMF.test(Ntestset)

In [ ]:

# def get_top_reco(predictions, userId, nam, rating, n=5):

hist_NMF_124, pred_NMF_124 = get_top_reco(predictions, 53932, interactions.name,interactions.rating)

AttributeError: 'DataFrame' object has no attribute 'name'